In [1]:
import os
import csv
import logging
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, CallbackContext, filters

In [2]:
# Set up logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# Create a directory for photos if it doesn't exist
if not os.path.exists('photos'):
    os.makedirs('photos')

In [5]:
# Define a function to handle incoming messages
async def handle_message(update: Update, context: CallbackContext) -> None:
    channel_title = update.effective_chat.title or update.effective_chat.username
    channel_username = update.effective_chat.username
    message_id = update.message.message_id
    sender = update.message.from_user.username
    timestamp = update.message.date.isoformat()
    
    # Initialize content structure
    message_content = update.message.text if update.message.text else ""
    media_paths = []

    # Check for images
    if update.message.photo:
        for photo in update.message.photo:
            file_id = photo.file_id
            new_file = await context.bot.get_file(file_id)
            image_path = f'photos/{file_id}.jpg'
            await new_file.download(image_path)
            media_paths.append(image_path)

    # Check for documents
    if update.message.document:
        document_file_id = update.message.document.file_id
        document_path = f'photos/{document_file_id}.pdf'
        new_doc = await context.bot.get_file(document_file_id)
        await new_doc.download(document_path)
        media_paths.append(document_path)

    # Prepare the structured data
    structured_data = {
        "Channel Title": channel_title,
        "Channel Username": channel_username,
        "ID": message_id,
        "Message": message_content,
        "Date": timestamp,
        "Media Path": ', '.join(media_paths)
    }

    # Append data to CSV file
    with open('telegram_data.csv', 'a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Channel Title', 'Channel Username', 'ID', 'Message', 'Date', 'Media Path']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if csvfile.tell() == 0:
            writer.writeheader()

        writer.writerow(structured_data)

    logger.info(f"Message from {sender}: {structured_data}")

# Define a function to start the bot
async def start(update: Update, _: CallbackContext) -> None:
    await update.message.reply_text('Bot started!')


In [6]:
def main():
    # Replace 'YOUR_TOKEN' with your Telegram Bot API token
    application = Application.builder().token("29294185:aab7ac1b04800f8cacd9bb582779349e").build()
    
    # Register handlers directly with the application
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT | filters.PHOTO | filters.Document, handle_message))

    # Start the Bot
    application.run_polling()

if __name__ == '__main__':
    main()

AttributeError: type object 'Document' has no attribute 'data_filter'